In [1]:
import intake
import logging
import cfunits
from collections import defaultdict
import xarray as xr
from typing import List, Union
import yaml

In [2]:
catalog = "https://data.nextgems-h2020.eu/online.yaml"
dataset = "ICON.ngc3028"
# replace as apppropriate

In [3]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)


def filter_warnings():
    import warnings

    warnings.filterwarnings(
        "ignore", "The return type of `Dataset.dims` will be changed to "
    )


filter_warnings()

In [23]:
with open("dataset_properties.yaml") as infile:
    expectations = yaml.safe_load(infile)

In [26]:
def test_metadata(ds: xr.Dataset, expectations: dict) -> List[str]:
    errors = []
    expected_vars = expectations["vars"]
    for var in expected_vars:
        if var not in ds:
            errors.append(f"{var} missing.")
            continue

        if (ret := test_units(ds, expected_vars, var)) is not None:
            errors.append(ret)

    if (
        "levels" in expectations
        and (ret := test_level_vars(ds, expectations=expectations["levels"]))
        is not None
    ):
        errors.append(ret)

    return errors


def test_level_vars(ds: xr.Dataset, expectations: dict):
    for name in expectations:
        if name not in ds:
            return f"dataset misses dimension variable {name}"
        level_values = list(ds[name].values)
        expected_values = expectations[name]["values"]
        if level_values != expected_values:
            return f"dimension {name} with values {level_values} does not match expectation of {expected_values}"


def test_units(ds, expected_vars, var):
    units = ds[var].attrs.get("units", None)
    if units is None:
        return f"{var} lacks units attribute. Should be {expected_vars[var]['units']}"

    elif not (
        cfunits.units.Units(units).equals(
            cfunits.units.Units(expected_vars[var]["units"])
        )
    ):
        return f"{var}'s units of '{units}' don't match expectation of '{expected_vars[var]['units']}'"


def test_values(ds: xr.Dataset, expectations: dict) -> List[str]:
    errors = []
    expected_vars = expectations["vars"]
    for var in expected_vars:
        if var not in ds:
            errors.append(f"{var} missing.")
            continue
        units = ds[var].attrs.get("units", "")
        print(
            f"{ds[var].min().values} {units} <= {var} <= {ds[var].max().values} {units}"
        )

        if not ds[var].min() >= expected_vars[var]["lower_bound"]:
            errors.append(
                f"ds[{var}] smaller than {expected_vars[var]['lower_bound']}: {ds[var].min().values}"
            )
        if not ds[var].max() <= expected_vars[var]["upper_bound"]:
            errors.append(
                f"ds[{var}] greater than {expected_vars[var]['upper_bound']}: {ds[var].max().values}"
            )
    return errors


def iterate_user_parameters(entry, fixed_keys: Union[dict, None] = None):
    if fixed_keys is None:
        fixed_keys = {}
    user_parameters = entry.describe()["user_parameters"]
    names = [x["name"] for x in user_parameters]
    remaining_keys = sorted([x for x in names if x not in fixed_keys.keys()])
    if len(remaining_keys) == 0:
        yield fixed_keys
        return
    param = user_parameters[len(fixed_keys)]
    logging.debug(f" iterating over {param}")
    key = param["name"]
    for value in param["allowed"]:
        fixed_keys[key] = value
        logging.debug(f"fixed keys for this iteration: {fixed_keys}")
        if len(remaining_keys) == 1:
            logging.debug(f"yielding {fixed_keys}")
            yield fixed_keys
        else:
            yield from iterate_user_parameters(entry, fixed_keys.copy())


def test_catalog_entry(
    entry, fixed_keys=None, test_function=lambda x: list(x), test_function_kwargs={}
):
    errors = defaultdict(list)
    for x in iterate_user_parameters(entry, fixed_keys=fixed_keys):
        logging.info(f"processing {x}")
        try:
            ds = entry(**x).to_dask()
        except KeyError as e:
            errors[str(x)].append(
                f"could not convert entry with params {x} to dask: KeyError: {e}"
            )
            continue
        except Exception as e:
            errors[str(x)].append(f"Error loading {x}: {e}")
        else:
            try:
                returns = test_function(ds, **test_function_kwargs)
                if returns:
                    errors[str(x)].extend(returns)
            except Exception as e:
                logging.critical(f"Error processing {x}: {e}")
                raise e
    if len(errors):
        raise BadDataset(errors)


class BadDataset(Exception):
    def __str__(self) -> str:
        error_string = ""
        errordict = self.args[0]
        for key, value in errordict.items():
            error_string += "\n" + "\n".join(f"{key}: {line}" for line in value) + "\n"
        return error_string

In [27]:
cat = intake.open_catalog(catalog)

test_catalog_entry(
    cat[dataset],
    fixed_keys=dict(),
    test_function=test_metadata,
    test_function_kwargs=dict(expectations=expectations["vars_3d"]),
)

INFO:root:processing {'time': 'PT30M', 'zoom': 7}
INFO:root:processing {'time': 'PT30M', 'zoom': 6}
INFO:root:processing {'time': 'PT30M', 'zoom': 5}
INFO:root:processing {'time': 'PT30M', 'zoom': 4}
INFO:root:processing {'time': 'PT30M', 'zoom': 3}
INFO:root:processing {'time': 'PT30M', 'zoom': 2}
INFO:root:processing {'time': 'PT30M', 'zoom': 1}
INFO:root:processing {'time': 'PT30M', 'zoom': 0}
INFO:root:processing {'time': 'PT3H', 'zoom': 7}
INFO:root:processing {'time': 'PT3H', 'zoom': 6}
INFO:root:processing {'time': 'PT3H', 'zoom': 5}
INFO:root:processing {'time': 'PT3H', 'zoom': 4}
INFO:root:processing {'time': 'PT3H', 'zoom': 3}
INFO:root:processing {'time': 'PT3H', 'zoom': 2}
INFO:root:processing {'time': 'PT3H', 'zoom': 1}
INFO:root:processing {'time': 'PT3H', 'zoom': 0}
INFO:root:processing {'time': 'P1D', 'zoom': 7}
INFO:root:processing {'time': 'P1D', 'zoom': 6}
INFO:root:processing {'time': 'P1D', 'zoom': 5}
INFO:root:processing {'time': 'P1D', 'zoom': 4}
INFO:root:proces

BadDataset: 
{'time': 'PT30M', 'zoom': 7}: Error loading {'time': 'PT30M', 'zoom': 7}: Failed to decode variable 'time': cannot reshape array of size 66240 into shape (64800,)

{'time': 'PT30M', 'zoom': 6}: cli missing.
{'time': 'PT30M', 'zoom': 6}: clw missing.
{'time': 'PT30M', 'zoom': 6}: hur missing.
{'time': 'PT30M', 'zoom': 6}: hus missing.
{'time': 'PT30M', 'zoom': 6}: qg missing.
{'time': 'PT30M', 'zoom': 6}: qr missing.
{'time': 'PT30M', 'zoom': 6}: qs missing.
{'time': 'PT30M', 'zoom': 6}: ta missing.
{'time': 'PT30M', 'zoom': 6}: ua missing.
{'time': 'PT30M', 'zoom': 6}: va missing.
{'time': 'PT30M', 'zoom': 6}: wa missing.
{'time': 'PT30M', 'zoom': 6}: zg missing.

{'time': 'PT30M', 'zoom': 5}: cli missing.
{'time': 'PT30M', 'zoom': 5}: clw missing.
{'time': 'PT30M', 'zoom': 5}: hur missing.
{'time': 'PT30M', 'zoom': 5}: hus missing.
{'time': 'PT30M', 'zoom': 5}: qg missing.
{'time': 'PT30M', 'zoom': 5}: qr missing.
{'time': 'PT30M', 'zoom': 5}: qs missing.
{'time': 'PT30M'

In [ ]:
test_catalog_entry(
    cat[dataset],
    fixed_keys=dict(zoom=0, time="P1D"),
    test_function=test_values,
    test_function_kwargs=dict(expectations=expectations["vars_3d"]),
)

INFO:root:processing {'zoom': 0, 'time': 'P1D'}


0.0 kg kg-1 <= cli <= 1.1850073860841803e-05 kg kg-1
0.0 kg kg-1 <= clw <= 0.00014487258158624172 kg kg-1
9.195958341479127e-07  <= hus <= 0.016969993710517883 
0.0 kg kg-1 <= qg <= 1.4504894352285191e-05 kg kg-1
0.0 kg kg-1 <= qr <= 1.5941172023303807e-05 kg kg-1
0.0 kg kg-1 <= qs <= 3.3542186429258436e-05 kg kg-1
163.17454528808594 K <= ta <= 299.22613525390625 K
-57.61587905883789 m s-1 <= ua <= 81.64440155029297 m s-1
-50.223899841308594 m s-1 <= va <= 39.78651809692383 m s-1


BadDataset: 
{'zoom': 0, 'time': 'P1D'}: hur missing.
{'zoom': 0, 'time': 'P1D'}: wa missing.
{'zoom': 0, 'time': 'P1D'}: zg missing.
